In [15]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# add url 
url = "https://www.zipcode.com.ng/2021/08/abuja-town-postal-code.html#garkia1"

In [158]:
url_codes = ['abuja-town', 'kwali-lga', 'abaji-lga']

In [ ]:
url_codes = ['abuja-town', 'kwali-lga', 'abaji-lga', 'kuje-lga', 'bwari-lga']

url = f"https://www.zipcode.com.ng/2021/08/{code}-code.html"

In [160]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [250]:

url_codes = ['abuja-town', 'kwali-lga', 'abaji-lga', 'bwari-lga', 'gwagwalada-lga', 'kuje-lga']

postal_codes_data = {}

for code in url_codes:
    
    url = f"https://www.zipcode.com.ng/2021/08/{code}-postal-code.html"
    
    response = requests.get(url)

    print(response.status_code)

    # Parse the HTML content using Beautiful Soup
    soup = BeautifulSoup(response.content, "html.parser")

    # Find the code-list div which contians the postal code data
    code_list_div = soup.find('div', class_ = 'table-responsive')
    
    if code_list_div is None:
        
        code_list_div = soup.find('div', class_ = 'code-list')
        
        items = code_list_div.find_all(['h3', 'li'])

        items[0].name

        if items[0].name == 'h3':
            print("#> webpage contains h3 tag")
            for item in items:
                if item.name == 'h3':
                    area = item.text
                    postal_codes_data[area] = []
                elif item.name == 'li':
                    street_name = item.find('div', class_ = 'listl').text
                    postal_code = item.find('div', class_ = 'listr').text
                    # append the data
                    postal_codes_data[area].append({'Street': street_name, "Postal Code": postal_code})
        else:
            print("#> webpage don't contain h3 tag")
            for item in items:
                street_name = item.find('div', class_='listl').text.strip()
                postal_code = item.find('div', class_='listr').text.strip()
                area_item = item.find_previous('div', class_='clist')
                if area_item:
                    area = area_item.text.strip()
                    # add data to the area data dictionary
                    if area not in postal_codes_data:
                        postal_codes_data[area] = []
                    postal_codes_data[area].append({'Street': street_name, 'Postal Code': postal_code})
    else:
        print("#> webpage is using table blocks")
        # Find the code-list div which contians the postal code data
        items = code_list_div.find_all(['tr'])

        for item in items:
            td_tags = item.find_all('td')[:3]
            if len(td_tags) == 3:
                street_name = td_tags[0].text.strip()
                postal_code = td_tags[1].text.strip()
                area = td_tags[2].text.strip()
                if area in postal_codes_data:
                    postal_codes_data[area].append({'Street': street_name, 'Postal Code': postal_code})
                else:
                    postal_codes_data[area] = [{'Street': street_name, 'Postal Code': postal_code}]

200
#> webpage don't contain h3 tag
200
#> webpage don't contain h3 tag
200
#> webpage don't contain h3 tag
200
#> webpage is using table blocks
200
#> webpage is using table blocks
200
#> webpage don't contain h3 tag


In [251]:
postal_codes_data.keys()

dict_keys(['Asokoro', 'Central Business District', 'Garki Area 1', 'Garki Area 2', 'Garki Area 3', 'Garki Area 7', 'Garki Area 8', 'Garki Area 10', 'Garki Area 11', 'Maitama', 'Three Arms Zone', 'Wuse II', 'Wuse Zone 1', 'Wuse Zone 2', 'Wuse Zone 3', 'Wuse Zone 4', 'Wuse Zone 5', 'Wuse Zone 6', 'Wuse Zone 7', 'Ashara', 'Dafa', 'Gumbo', 'Kilankwa', 'Kwali', 'Pai', 'Yangoji', 'Yebu', 'Wako', 'Abaji', 'Yaba', 'Bwari', 'Gwagwalada', 'Kuje', 'Rubbochi'])

In [252]:
data = []

In [253]:
for area, streets, in postal_codes_data.items():
    for street_info in streets:
        data.append({'Area': area, 'Street': street_info['Street'], 'Postal Code': street_info['Postal Code']})


In [254]:
df = pd.DataFrame(data)

In [255]:
df.replace(to_replace='St.', value='Street', regex=True, inplace=True)
df.replace(to_replace='Cr.', value='Cresent', regex=True, inplace=True)

In [256]:
len(df)

839

In [257]:
df

,Area,Street,Postal Code
0,Asokoro,Ajayi Cresentwder Street,900231
1,Asokoro,Akenzua Street,900231
2,Asokoro,Ali Akihi Cresent,900231
3,Asokoro,Anwar Sadat Street,900231
4,Asokoro,Dennis Osadebe Street,900231
...,...,...,...
834,Rubbochi,Yaba,903102
835,Rubbochi,Yewusa,903102
836,Rubbochi,Zagabutu,903102
837,Rubbochi,Zoge,903102


In [258]:
# exporting dataframe
df.to_csv('../data/fct-postal-code.csv')

In [260]:
import json

json_df = json.dumps(postal_codes_data, indent=4)

with open('../data/fct-postal-codes2.json','w') as json_file:
    json_file.write(json_df)